# Analysis 1

### Lauren Lunsford, Ye Chan Kim, Christopher Peña 
#### Date Started: February 3, 2022
#### Last Updated: February 4, 2022

**Questions:** 
- Should we use NaN or None?
- How to factor in qty. Does the qty 

In [ ]:
#importing packages
import pandas as pd

## 1. Loading and Viewing Data

In [ ]:
wd='UMich_Sample_Data/'
b=pd.read_csv(wd+'UMich-SampleData-PicksMadePNW-Jan3.csv')

Note:
- TIMESTAMP_UTC appears to be whenever that record was made.
- BOXIN/BOXOUT are when the order was started/completed
- PICKORDER…. Appears to be the same as BOXIN/BOXOUT
- PICKCOMPLETE is when the button was pressed
- PICKORDERSUBMITTIME is when the order was uploaded to the server
- PICKORDERENDTIME is when the full order was completed (if >1 box, when all boxes are completed)

### 1.2 Picks Made PNW Jan3


In [ ]:
b.head()

In [ ]:
b.columns

## 2. Analysis 1: unit hours per product packed

In [ ]:
df=b[['PICKCOMPLETETIME','BOXBARCODE','PRODUCTNAME']]
df.head(5)

In [ ]:
def combine(row):
    pname=row['PRODUCTNAME']
    time=row['PICKCOMPLETETIME']
    return [pname, time]
    

In [ ]:
df['concat']=df.apply(lambda x: combine(x), axis=1)

In [ ]:
df=df[['BOXBARCODE','concat']]
df.head()

In [ ]:
df=df.groupby('BOXBARCODE')['concat'].apply(list)
df=df.reset_index(name='[Product, PickTime]')
df.head()

In [ ]:
from datetime import datetime

t0=df['[Product, PickTime]'][1][0][1]
t1=df['[Product, PickTime]'][2][0][1]
t0=datetime.strptime(t0, '%Y-%m-%d %H:%M:%S')
t1=datetime.strptime(t1, '%Y-%m-%d %H:%M:%S')
t1-t0

In [ ]:
def get_time_diff(lst):
    output_lst=[]
    #making second item in each tuple a date time so we can sort it accordingly
    for tup in lst:
        from datetime import datetime
        tup[1] = datetime.strptime(tup[1], '%Y-%m-%d %H:%M:%S')
    
    #sort product,picktime by picktime within list
    lst=sorted(lst, key=lambda x: x[1])
    
    #turning l=[(p1,t1),(p2,t2),(p3,t3)]
    #into new_l=[(p1,t0-t1),(p2,t1-t2),(p3,t2-t3)]
    
    for i in range(len(lst)):
        if i==0:
            time_delta=None  #IDK IF I SHOULD DO NONE OR NaN 
        else:
            t1=lst[i][1]
            t0=lst[i-1][1]
            time_delta=t1-t0
        output_lst.append([lst[i][0],time_delta])
    return output_lst
    

In [ ]:
product_times={}
for product_picktime in df['[Product, PickTime]']:
    product_pickdiffs=get_time_diff(product_picktime)
    for product, pickdiff in product_pickdiffs:
        if product in product_times.keys():
            product_times[product].append(pickdiff)
        else:
            product_times[product]=[pickdiff]

In [ ]:
def mean(lst):
    summ=0
    count=0
    for num in lst:
        if num==None:
            pass
        else:
            num=num.total_seconds()
            summ+=num
            count+=1
    return summ/count

In [ ]:
product_time_averages= []
for product in list(product_times.keys()):
    product_time_averages.append((product,mean(product_times[product])))
product_time_averages=sorted(product_time_averages, key=lambda x:x[1])
product_time_averages

In [ ]:
print( len(product_time_averages), len(b['PRODUCTNAME'].unique()))
print("Need to find the 2 values. My guess is they're both ")

### 2.5 Cold Chain times

Calculating cost of CC items includes:
   
   - Time to pack
   - How much of the item you can fit into a box before you need to add the extra packaging (so packaging cost / %space of box it takes up)

In [ ]:
CC_product_time_averages=[]
for item in product_time_averages:
    if item[0][4:6]=='CC':
        CC_product_time_averages.append(item)

In [ ]:
CC_product_time_averages

## 3. Exploration of Results

### 3. A) Value Distribution

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 1)

# We can set the number of bins with the *bins* keyword argument.
axs.hist(ptimes, bins=50)

There's a fun little logistic distribution

### 3. B) Correlation Matrix

Need: ZONENAME	SHELF	LEVEL for each product

In [ ]:
#checking ot see if shelves, zones, and levels are the same for each product
zonenames=[]
shelves=[]
levels=[]
for product in b['PRODUCTNAME'].unique():
    x=b[b['PRODUCTNAME']==product]
    dct=x.to_dict('list')
    zonenames.append(len(set(dct['ZONENAME'])))
    shelves.append(len(set(dct['SHELF'])))
    levels.append(len(set(dct['LEVEL'])))
print(np.mean(zonenames), np.mean(shelves), np.mean(levels))


This means that the zones and shelves are constant, but sometimes, the levels where products are change

In [ ]:
b.head()

In [ ]:
b.columns

In [ ]:
c=b[['QTY']]

## I. Irrelevant Code

This stuff basically creates new columns where one column is the order and the other column is which box in the order it is 

In [ ]:
def get_box(BOXBARCODE):
    parts=BOXBARCODE.split('_')
    if len(parts)==3:
        return parts[2]
    else:
        return ''
    

In [ ]:
df['order']=df['BOXBARCODE'].apply(lambda x: x.split('_')[1] )
df['box']=df['BOXBARCODE'].apply(lambda x: get_box(x) )
df.head()